# Jeffries Human Brain Aging - pySCENIC pipeline (Embedded version)

**Author:** Vincent Gardeux

**Date Created:** 2025-11-17

**Date Modified:** 2025-11-21

In [1]:
# Fix OPENBLAS Warnings
import os
param_n_workers = 24 # We have 112 CPUs/cores, each process will automatically be associated to a different CPU by the OS scheduler
param_threads_per_worker=1 # We have 2 threads per CPU on SVEN (hyper-threading). See lscpu command. Note: Here they are not used apparently. Setting to 1 or 2 gives similar c. time
os.environ['OPENBLAS_NUM_THREADS'] = f"{param_n_workers * param_threads_per_worker}"
os.environ['MKL_NUM_THREADS'] = f"{param_n_workers * param_threads_per_worker}"
os.environ['OMP_NUM_THREADS'] = f"{param_n_workers * param_threads_per_worker}"
os.environ['NUMEXPR_MAX_THREADS'] = f"{param_n_workers * param_threads_per_worker}"

# import dependencies
import pandas as pd
import numpy as np
import anndata as ad
import regdiffusion as rd # For replacing grnboost2 which is slow as hell and bugs (stalls) when there are more than ~1.5B elements in the matrix
import ast # For reading frozenset as strings
import pickle
import pytz

from datetime import datetime
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from distributed import Client, LocalCluster

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.binarization import binarize
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

In [2]:
# [Input] H5ad file to use
EXPRESSION_H5AD_FNAME = '/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pfc.clean.h5ad' # Built from pfc.clean.rds. Downloaded from https://publications.wenglab.org/SomaMut/Jeffries_Yu_BrainAging_2025/

# [Input] Transcription factors list (SCENIC step 1: GRNBoost2)
f_tfs = "/data/gardeux/Neuro_Droso_ND75KD/data/allTFs_hg38.txt" # From https://resources.aertslab.org/cistarget/tf_lists/
# Derive list of Transcription Factors(TF)
tf_names = load_tf_names(f_tfs)

# [Output] Adjacency matrix (SCENIC step 1: GRNBoost2)
adj_matrix = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_adj.csv"

# [Input] Ranking databases (SCENIC step 2-3: cisTarget)
f_db_names = ["/data/gardeux/Neuro_Droso_ND75KD/data/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather", "/data/gardeux/Neuro_Droso_ND75KD/data/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather"] # From pySCENIC db: https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/
dbs = [RankingDatabase(fname=f_name, name=os.path.basename(f_name)) for f_name in f_db_names]

# [Input] Motif databases (SCENIC step 2-3: cisTarget)
f_motif_path = "/data/gardeux/Neuro_Droso_ND75KD/data/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl" # From pySCENIC db: https://resources.aertslab.org/cistarget/motif2tf/

# [Output] Regulons (SCENIC step 2-3: cisTarget)
f_motifs_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_motifs.tsv"
f_modules_pickle = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_modules.pkl"
f_modules_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_modules.tsv"
f_regulons_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_regulons.tsv"
f_regulons_pickle = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_regulons.pkl"
f_regulons_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_regulons_aucell.tsv"
f_regulons_binarized_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_regulons_aucell_binarized.tsv"
f_regulons_binarization_thresholds_aucell_path = "/data/gardeux/Neuro_Droso_ND75KD/data/Jeffries_2025_HumanBrainAging/pySCENIC/Jeffries_HBAging_regulons_aucell_binarization_thresholds.tsv"

Load ex_matrix

In [ ]:
# [Input] Load expression matrix from H5ad file
f_h5ad = ad.read_h5ad(EXPRESSION_H5AD_FNAME)
f_gene_names = f_h5ad.var_names.tolist()  # Gene names
f_cell_names = f_h5ad.obs_names.tolist()   # Cell names
ex_matrix = pd.DataFrame.sparse.from_spmatrix(f_h5ad.X.T, index=f_gene_names, columns=f_cell_names)

# Restrict matrix to feather genes
ranking_feather = pd.read_feather(f_db_names[0])
overlap_values = ex_matrix.index[pd.Series(ex_matrix.index).isin(ranking_feather.columns)].unique()
ex_matrix = ex_matrix.loc[overlap_values, :] # This step takes forever

ex_matrix # 18099 genes x 367317 cells

In [ ]:
# Safety check
(f_h5ad.obs.index == ex_matrix.columns).all()

# SCENIC steps

## STEP 1: Gene regulatory network inference, and generation of co-expression modules

### 1.a. GRN inference using the GRNBoost2 algorithm

In the initial phase of the pySCENIC pipeline the single cell expression profiles are used to infer co-expression modules from.

Run GRNboost from arboreto to infer co-expression modules

The arboreto package is used for this phase of the pipeline.

*Output:* List of adjacencies between a TF and its targets.



**Option 1:** Run GRNBoost2 algorithm

```
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

# Prepare the multithreading
cluster = LocalCluster(name='grn_call', dashboard_address=":12345", n_workers=param_n_workers, threads_per_worker=param_threads_per_worker)
client = Client(cluster)

# Here I run the function within the package (no CLI)
adjacencies = grnboost2(expression_data=ex_matrix.transpose(), tf_names=tf_names, seed=42, verbose=True, client_or_address=client)
            
# Shutting down cluster
client.close()
cluster.close()
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~50mn with n_workers=12, threads_per_worker=2 => With reduced matrix to overlapping genes between .feather file and our matrix
```

Here it stalls for several days at step "creating dask graph"...

I think there is a bug in grnboost2 which makes it incompatible with big matrices i.e. impossible to deal with >~1.5B elements in the matrix. Here we have 6.6B...

So I'll use regdiffusion instead.

**Option 2:** Run regdiffusion specific code
### * * REGDIFFUSION SPECIFIC CODE * * ###

Run RegDiffusionTrainer instead of the (slow) GRNBoost2 algorithm. See https://tuftsbcb.github.io/RegDiffusion/downstream_with_pyscenic.html

In [ ]:
# Prepare dataset for RegDiffusionTrainer (needs to be logged, cells as rows)
ex_matrix_log = np.log(ex_matrix.transpose() + 1.0) # Transpose and log
ex_matrix_log = ex_matrix_log.loc[:, ~(ex_matrix_log == 0).all()] # Drop columns where all values are 0
ex_matrix_log

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

rd_trainer = rd.RegDiffusionTrainer(ex_matrix_log.to_numpy(), device="cpu")
rd_trainer.train()

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~2h38 with n_workers=24, threads_per_worker=1

Extract edges from GRN

In [ ]:
# Now we focus on edges with weight > 50 percentile. 
grn = rd_trainer.get_grn(ex_matrix_log.columns, top_gene_percentile = 50) # gene_names to recover non-expressed genes

# Here for each gene, we are going to extract all edges
adjacencies = grn.extract_edgelist(k = -1, workers = param_n_workers)
adjacencies.columns = ['TF', 'target', 'importance']

# check edgelist.  
adjacencies

Read in the adjacencies matrix

In [ ]:
# Restrict to tf names
adjacencies = adjacencies[adjacencies['TF'].isin(tf_names)]
# Sort by importance
adjacencies = adjacencies.sort_values(by='importance', ascending=False)
adjacencies

### * * / REGDIFFUSION SPECIFIC CODE * * ###

Here is the end of the regdiffusion specific code. If grnboost2 worked, then you can resume here.

In [ ]:
print(adjacencies.TF.nunique(), "unique TF-modules were found ( out of",len(tf_names),").")

In [ ]:
print(adjacencies.target.nunique(), "unique targets were found ( out of",len(ex_matrix.index),").")

In [ ]:
adjacencies.TF.isin(tf_names).all()

In [ ]:
adjacencies.to_csv(adj_matrix, index=False, sep=',')

In [ ]:
# Checkpoint to recover from file (eventually)
#adjacencies = pd.read_csv(adj_matrix, sep=',', na_filter=False) # If na_filter=True, the nan gene is detected as NaN
adjacencies

## STEP 2-3: Regulon prediction aka cisTarget

*Output:* List of adjacencies between a TF and its targets.

### 2.a. Running regulon prediction using cisTarget

Here, we use the --mask_dropouts option, which affects how the correlation between TF and target genes is calculated during module creation. It is important to note that prior to pySCENIC v0.9.18, the default behavior was to mask dropouts, while in v0.9.18 and later, the correlation is performed using the entire set of cells (including those with zero expression). When using the modules_from_adjacencies function directly in python instead of via the command line, the rho_mask_dropouts option can be used to control this.

**Note:** In the following code, I use `rho_mask_dropouts=True` for keeping the old R behaviour. It also produces slightly more regulons.

<span color="red">**Issue:** This function `modules_from_adjacencies` is inherently bugged. As it's supposed to run on a single thread but bypass all my specifications to run on all available cores. I don't know how to fix this behaviour and for the function to limit the number of cores</span>

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

modules = list(modules_from_adjacencies(adjacencies, ex_matrix.transpose(), rho_mask_dropouts=True, keep_only_activating=True))

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~48mn with n_workers=24, threads_per_worker=1. BUT USES ALL 112 cores!!

In [ ]:
modules_df = pd.DataFrame(index = range(0, len(modules)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Context", "NES", "Markers"))
for j in range(0, len(modules)):
    # Setting values
    context = list(modules[j].context)
    modules_df["Regulon"].iloc[j] = modules[j].name
    modules_df["TF"].iloc[j] = modules[j].transcription_factor
    modules_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    modules_df["NbMarkers"].iloc[j] = len(set(modules[j].gene2weight))
    modules_df["Context"].iloc[j] = context[1]
    modules_df["NES"].iloc[j] = modules[j].score
    modules_df["Markers"].iloc[j] = ','.join(list(modules[j].gene2weight))

modules_df = modules_df.sort_values(by='NbMarkers', ascending=False)
modules_df.to_csv(f_modules_path, index=False, sep = "\t")
modules_df

In [ ]:
print(modules_df.TF.nunique(), "unique TF-modules were found ( out of",len(tf_names),"). Modules with less than 20 markers were filtered out.")

**Note:** In the following code, I use `filter_for_annotation=False` for deactivating the pruning/filtering and producing all possible regulons. This may not be the more conservative behaviour, so feel free to deactivate it with `filter_for_annotation=True` and then play with the filtering parameters: 
- `weighted_recovery=False`
- `rank_threshold=1500`
- `nes_threshold=3`
- `motif_similarity_fdr=0.001`
- `auc_threshold=0.05`

Of course, even if these parameters are set, they will have no impact on the result if `filter_for_annotation=False`

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

df = prune2df(dbs, modules, f_motif_path, num_workers=param_n_workers, weighted_recovery=False, rank_threshold = 1500, nes_threshold=3, motif_similarity_fdr=0.001, auc_threshold=0.05, filter_for_annotation=False)
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~40mn with n_workers=24, threads_per_worker=1.

In [ ]:
df.to_csv(f_motifs_path, sep = "\t")
df

In [ ]:
# # Checkpoint
# # Reading back the data from tsv
# ## 1. Read with multi-index headers
# df = pd.read_csv(f_motifs_path, sep="\t", header=[0, 1], index_col=[0, 1])
# ## 2. Transform the "Context" frozensets (string) into actual frozensets
# def parse_frozenset_string(s):
#     if isinstance(s, str) and s.startswith('frozenset'):
#         return frozenset(ast.literal_eval(s[len('frozenset('):-1]))
#     return s
# df.loc[:, ('Enrichment', 'Context')] = df.loc[:, ('Enrichment', 'Context')].apply(parse_frozenset_string)
# ## 3. Transform "TargetGenes" from string to list of tuples
# def parse_list_of_tuples(s):
#     if isinstance(s, str) and s.startswith('[') and s.endswith(']'):
#         return ast.literal_eval(s)
#     return s  # if it's already a list or something else
# df.loc[:, ('Enrichment', 'TargetGenes')] = df.loc[:, ('Enrichment', 'TargetGenes')].apply(parse_list_of_tuples)

In [ ]:
print(len(set(df.index.get_level_values('TF').values)), "regulons were kept, after pruning")

In [ ]:
# Look for main regulons I'm looking for
print("ATF4", "ATF4" in df.index.get_level_values('TF').values, sep="\t")

In [ ]:
print("Size of Dataframe:", len(df))
# Check which rows have empty lists in 'TargetGenes'
mask = df[('Enrichment', 'TargetGenes')].apply(lambda x: len(x) == 0)
# Drop those rows
df_filtered = df.loc[~mask]
print("Size of Dataframe:", len(df_filtered))

These "modules" are then combined into regulons, by taking the top NES for each TF (for main Motif, and final score of regulon). All genes are bundled together.

In [ ]:
# This dataframe can then be converted to regulons.
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

regulons = df2regulons(df_filtered)

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~12mn with n_workers=24, threads_per_worker=1.

In [ ]:
regulon_df = pd.DataFrame(index = range(0, len(regulons)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Motif", "NES", "Markers"))
for j in range(0, len(regulons)):
    # Fixing order of set
    context = list(regulons[j].context)
    if(context[0].endswith(".png")):
        tmp = context[0]
        context[0] = context[1]
        context[1] = tmp
    # Setting values
    regulon_df["Regulon"].iloc[j] = regulons[j].name
    regulon_df["TF"].iloc[j] = regulons[j].transcription_factor
    regulon_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    regulon_df["NbMarkers"].iloc[j] = len(set(regulons[j].gene2weight))
    regulon_df["Motif"].iloc[j] = "https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/logos/" + context[1]
    regulon_df["NES"].iloc[j] = regulons[j].score
    regulon_df["Markers"].iloc[j] = ','.join(list(regulons[j].gene2weight))

regulon_df = regulon_df.sort_values(by='NbMarkers', ascending=False)
regulon_df.to_csv(f_regulons_path, index=False, sep = "\t")
regulon_df

In [ ]:
# Look for main regulons
print("ATF4", "ATF4" in regulon_df.TF.values, sep="\t")

## Phase III: Cellular regulon enrichment matrix (aka AUCell)

Characterize the different cells in a single-cell transcriptomics experiment by the enrichment of the regulons. Enrichment of a regulon is measures as AUC of the recovery curve of the genes that define this regulon.

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

auc_mtx = aucell(ex_matrix.transpose(), regulons, num_workers=param_n_workers)

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: Takes ~1h13 with n_workers=24, threads_per_worker=1.

In [ ]:
auc_mtx.to_csv(f_regulons_aucell_path, sep = "\t")
auc_mtx

In [ ]:
# Checkpoint to regenerate the object from the file
#auc_mtx = pd.read_csv(f_regulons_aucell_path, sep = "\t", index_col = "Cell")
#auc_mtx.columns.name = "Regulon"
#auc_mtx

In [ ]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

auc_mtx_bin = binarize(auc_mtx, seed = 42, num_workers=param_n_workers)

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: 08h40mn46 with num_workers=32

In [ ]:
import numpy as np

# columns with all-constant values
constant_cols = [c for c in auc_mtx.columns 
                 if np.allclose(auc_mtx[c].values, auc_mtx[c].values[0])]

print("Number of constant columns:", len(constant_cols))

In [ ]:
binarization_thresholds = auc_mtx_bin[1]
binarization_thresholds.to_csv(f_regulons_binarization_thresholds_aucell_path, sep = "\t")
binarization_thresholds

In [ ]:
auc_mtx_bin = auc_mtx_bin[0]
auc_mtx_bin.to_csv(f_regulons_binarized_aucell_path, sep = "\t")
auc_mtx_bin

In [ ]:
sum(auc_mtx_bin["ATF4(+)"])

In [ ]:
binarization_thresholds.loc['ATF4(+)']

In [ ]:
sum(auc_mtx["ATF4(+)"] > binarization_thresholds.loc['ATF4(+)'])